 Lista de sites (roots):

* https://www.amazon.com.br/
* https://www.americanas.com.br/ <- resolver timeout
* https://www.bigboygames.com.br/
* https://www.mercadolivre.com.br/
* https://www.magazineluiza.com.br/
* https://www.submarino.com.br/
* https://www.buscape.com.br/
* https://www.maurospbrgames.com.br/ <- pode ser tirado dependendo da situação
* https://shopee.com.br/
* https://jogosbarato.com/ <- possuie as informções mas sem ser em tabela
* https://www.kabum.com.br/  <- possuie as informções mas sem ser em tabela
* https://www.tcagames.com.br/  <- possuie as informções mas sem ser em tabela
* https://lojaarenagames.com.br/ 
* https://www.shockgames.com.br/
* https://www.baraogames.com/
* https://www.xplacegames.com.br/
* https://www.futuristicgames.com.br/
* https://www.vnsgames.com.br/
* https://www.pontofrio.com.br/
* https://www.ibyte.com.br/
* https://www.extra.com.br/
* https://www.fastshop.com.br/

In [ ]:
!pip install beautifulsoup4
!pip install validators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19582 sha256=1599759e49721a0c187584c446308abedb2a606e4798ceb35ddf1232b14e6c08
  Stored in directory: /root/.cache/pip/wheels/5f/55/ab/36a76989f7f88d9ca7b1f68da6d94252bb6a8d6ad4f18e04e9
Successfully built validators


In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from threading import Thread
import urllib.robotparser as urobot
import urllib.error
import re
import socket
import validators
import time


sites = [
            "https://www.bigboygames.com.br/",
            "https://www.magazineluiza.com.br/",
            # "https://www.kabum.com.br/", --> resolver conecção
            # "https://lojaarenagames.com.br/", --> parser robot.txt com negando todos os links
            "https://www.shockgames.com.br/",
            # "https://www.futuristicgames.com.br/", --> parser robot.txt dando problema
            "https://www.vnsgames.com.br/",
            "https://www.ibyte.com.br/",
            # "https://www.extra.com.br/", -->resolver timeout
            # "https://www.fastshop.com.br/" --> parser robot.txt com negando todos os links
        ]

delay = 1 #em segundos
qtpaginas = 5



#fronteira
class Frontier:
    def __init__(self, root):
        self.list = [root]
        self.looked = set()

    def get_next_url(self):
        nxt = self.list.pop(0)
        while nxt in self.looked:
            nxt = self.list.pop(0)
        self.looked.add(nxt)
        return nxt

    def add_urls(self, urls):
        self.list += urls

#pega a pagina html
def get_html_page(url):
    if validators.url(url):
        try:
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            page = urlopen(req, timeout=5)
        except urllib.error.HTTPError as err:
            # save the url an the erro but still running
            print('http-error')
            return None
        except urllib.error.URLError as err:
            print("url-error")
            # save the url an the erro but still running
            return None
        except socket.timeout:
            print("socket timeout")
        else:
            if page.headers.get_content_type() =="text/html":
                return BeautifulSoup(page, "html.parser")
            else:
                return None
    else: 
        return None


#extrai os links da pagina para a fronteira
def extract_links(page, root, robot):
    robot.read()
    links = []
    for link in page.findAll('a', href=True):
        href = link.get('href')
        if not re.search("^https:", href):
            href = root + href
        if re.search(rf"^{root}", href):
            if robot.can_fetch("*", href):
                links.append(href)
    return links


#salva a pagina html

def save_page(page, root, cont):
      print(root + ": "+ cont)
#     soup = str(page)
#     site_name = root[8:len(root)-1]
#     Path('./' + site_name).mkdir(parents=True, exist_ok=True)
#     html_file = open(site_name + "/" + cont + ".html", "w")
#     html_file.write(soup)
#     html_file.close()
#     print(".")



#bot do crawler
class Bot:
    def __init__(self, root):
        self.root = root
        self.frontier = Frontier(root)
        self.robot = urobot.RobotFileParser()
        self.robot.set_url(root + 'robots.txt')

    def run(self, delay, n_pages):
        count = 1
        while n_pages > 0:

            url = self.frontier.get_next_url()
            page = get_html_page(url)
            if page is not None:
                urls = extract_links(page, self.root, self.robot)
                self.frontier.add_urls(urls)
                save_page(page, url, str(count))
                count += 1
                n_pages -= 1
            time.sleep(delay)


def main():
    bot = None
    for root in sites:
        bot = Bot(root)
        bot.run(delay,qtpaginas)


if __name__ == "__main__":
    main()

https://www.fastshop.com.br/: 1


IndexError: ignored